In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


## Load the trainmodel , onehot encoder, and scalar pickle

In [4]:
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scalar.pkl','rb') as file:
    scalar = pickle.load(file)
    
with open('onehot_encode_geo.pkl', 'rb') as file:
    onehot_encode_geo = pickle.load(file)
    


In [18]:
## Example input data

input_data = {
	'creditScore' : 600,
	'Geography' : 'France',
	'Gender' : 'Male',
	'Age' : 40,
	'Tenure' : 3,
	'Balance' : 60000,
	'NumOfProducts' : 2,
	'HasCrCard' : 1,
	'IsActiveMember' : 1,
	'EstimatedSalary' : 50000
}

In [9]:
geo_encoded = onehot_encode_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Niru\Desktop\ANN clasiification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [10]:
input_df = pd.DataFrame([input_data])
input_df

,creditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
##Encode categorical variables

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,creditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [13]:
## concatenation with onehot encoded data


input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df,],axis=1)
input_df

,creditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
input_df.columns = [col.replace("creditScore", "CreditScore") for col in input_df.columns]


In [21]:
##scaling the data

scaled_input = scalar.transform(input_df)
scaled_input

array([[-0.55012981,  0.89091075,  0.12136034, -0.68041201, -0.28051905,
         0.84584804,  0.62670381,  0.968496  , -0.91572441,  1.00400803,
        -0.57427105, -0.58350885]])

In [22]:
prediction = model.predict(scaled_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step


In [23]:
prediction

array([[0.04229403]], dtype=float32)

In [24]:
prediction_proba = prediction[0][0]

In [25]:
prediction_proba

np.float32(0.042294033)

In [27]:
if prediction_proba > 0.5 :
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
